In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv


/home/siyer5/GNN-for-DCT/.venv/lib/python3.8/site-packages/torch/__config__.py:10: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11070). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._show_config()


In [129]:
torch.cuda.is_available()

False

In [124]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data, DataLoader

# Step 1: Generate a 1D Lattice Graph with 3 neighbors on each side
def create_1d_lattice_graph(lattice_size=149, neighbors=3):
    edges = []
    for i in range(lattice_size):
        edges.append([i, i])  # self loop
        for j in range(1, neighbors + 1):
            edges.append([i, (i - j) % lattice_size])  # Connect to left neighbors
            edges.append([i, (i + j) % lattice_size])  # Connect to right neighbors

    edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    return edge_index

# Step 2: Define the GNN Model
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        # First GCN layer + ReLU activation
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        # Second GCN layer
        x = self.conv2(x, edge_index)
        return x

def generate_data(lattice_size=149, sample_size=100000, prob=0.5):
    if prob < 0 or prob > 1:
        raise ValueError('Probability must be between 0 and 1.')
    
    edge_index = create_1d_lattice_graph(lattice_size=lattice_size)
    data_list = []
    
    for _ in range(sample_size):
        if prob == 0.5:
            # Generate random input states for each node
            x = torch.randint(0, 2, (lattice_size, 1), dtype=torch.float)
        elif prob == 0.0:
            # Generate all zeros
            x = torch.zeros(lattice_size, 1)
        elif prob == 1.0:
            # Generate all ones
            x = torch.ones(lattice_size, 1)
        else:
            x = torch.bernoulli(prob * torch.ones(lattice_size, 1))
        
        # Define the binary density classification based on the mean of cell states
        y = (x.mean() > 0.5).long().repeat(lattice_size)  # 0 for < 0.5 density, 1 for >= 0.5, repeated to match lattice size
        
        # Create a Data object and append to the list
        data_list.append(Data(x=x, edge_index=edge_index, y=y))
    
    return data_list

# Step 3: Create the data and model
lattice_size = 149
prob = 0.4
train_data_list = generate_data(lattice_size=lattice_size, sample_size=10000, prob=prob)
test_data_list = generate_data(lattice_size=lattice_size, sample_size=10000, prob=prob)

train_loader = DataLoader(train_data_list, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data_list, batch_size=32, shuffle=False)

# Define model, optimizer, and loss function
model = GCN(input_dim=1, hidden_dim=16, output_dim=2)  # 2 for binary classification
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Step 4: Training Loop
def train(loader, model, optimizer, criterion, epochs=400):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for data in loader:
            optimizer.zero_grad()
            out = model(data.x, data.edge_index)  # Forward pass
            loss = criterion(out, data.y)         # Compute the loss
            loss.backward()                       # Backpropagation
            optimizer.step()                      # Update the parameters
            total_loss += loss.item()
        print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(loader)}')

# Step 5: Testing Loop
def test(loader, model):
    model.eval()
    correct = 0
    for data in loader:
        out = model(data.x, data.edge_index)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    accuracy = correct / (len(loader.dataset) * lattice_size)
    print(f'Accuracy: {accuracy:.4f}')

# Train the model
train(train_loader, model, optimizer, criterion, epochs=20)

# Test the model
test(test_loader, model)

Epoch 1/20, Loss: 0.07761598925906629
Epoch 2/20, Loss: 0.04245873135380661
Epoch 3/20, Loss: 0.041115974884676616
Epoch 4/20, Loss: 0.03986991295433297
Epoch 5/20, Loss: 0.03960084149497933
Epoch 6/20, Loss: 0.038599927479716634
Epoch 7/20, Loss: 0.03855370053018744
Epoch 8/20, Loss: 0.03854553589371208
Epoch 9/20, Loss: 0.03828411595152018
Epoch 10/20, Loss: 0.03839492891355754
Epoch 11/20, Loss: 0.03838525237158512
Epoch 12/20, Loss: 0.03772421073936188
Epoch 13/20, Loss: 0.038429872358229734
Epoch 14/20, Loss: 0.03926965823105444
Epoch 15/20, Loss: 0.03827127118404514
Epoch 16/20, Loss: 0.03843506221799329
Epoch 17/20, Loss: 0.038489515361295766
Epoch 18/20, Loss: 0.03827760990682287
Epoch 19/20, Loss: 0.0386010192069835
Epoch 20/20, Loss: 0.0385254935462611
Accuracy: 0.9931


In [127]:
sample_test_data = generate_data(lattice_size=lattice_size, sample_size=10000, prob=0.6)
sample_test_loader = DataLoader(sample_test_data, batch_size=32, shuffle=False)
test(sample_test_loader, model)

Accuracy: 0.0066


In [44]:
sample = generate_data(lattice_size=149, sample_size=1)
sample[0].y

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1])